# Data Collection for Aboveground Biomass 

The analysis is split between 3 notebooks.:
1) data_collection.ipynb (this notebook): which downloads data from Google Earth Engine (can be slow)
2) data_processing.ipynb: which merges the data into dataframe that is ready to use for regression
3) regression.ipynb: which performs the regression and prediction of the Aboveground Biomass and calculates the carbon stored

In this notebook, we:
* define our *area of interest* (AOI) and the *time* of interest
* connect to Google Earth Engine to download data 
* access sentinel-2 data (optical), sentinel-1 data (SAR), elevation and land cover data
* resample all data to common grid
* download data

## Import Libraries

All libraries should be easily installed via: 

`!pip install [module]`

In [62]:
import numpy as np
import os

import ee
import geemap
import datetime

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize()


## Input

In [63]:
#Set the date from which you want the collect data 
date_input = '2022-01-01'

# Choose a good projection for your area of interest (Google whats the best option)
projection_at_aoi = 'EPSG:3857' # This works well for Germany

Set the area of interest.

This can be set at [https://geojson.io/](https://geojson.io/)

In [64]:
# Input a geojson for the area of interest
aoi_geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              10.327876967233976,
              51.190950056438965
            ],
            [
              10.327876967233976,
              51.02766008862861
            ],
            [
              10.544399323239077,
              51.02766008862861
            ],
            [
              10.544399323239077,
              51.190950056438965
            ],
            [
              10.327876967233976,
              51.190950056438965
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [65]:
# Preparing the aoi for different use cases
aoi_input = aoi_geojson['features'][0]['geometry']['coordinates'][0]
center_aoi = (np.array(aoi_input[0]) + np.array(aoi_input[2]))/2
aoi = ee.Geometry.Polygon(aoi_input) 
geometry = ee.Geometry.Polygon(aoi_input,None, False)

## Data Download

In this section, the data from different sources is downloaded

### Sentinel 2 Data Download

For Sentinel-2 Data, we need to make sure to create a cloud free image. We use the recommend approach from Google Earth Engine

In [66]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select(['B2', 'B3', 'B4','B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12','SCL']).updateMask(not_cld_shdw)

In [67]:
# ---- PARAMETERS ----
# Define your AOI (a rectangle in this case)

# Define date range
end_date_str = date_input
end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

# Range of data collection
n_days = 365
start_date = end_date - datetime.timedelta(days=n_days)
# Convert back to string for GEE
start_date_str = start_date.strftime('%Y-%m-%d')

# This parameter is used to filter the image collection by cloud cover percentage.
# The value is set to 60, meaning that only images with a cloud cover percentage of 60% or less will be included in the collection.
CLOUD_FILTER = 60
# This parameter is used to set the threshold for cloud probability.
# The value is set to 40, meaning that pixels with a cloud probability of 40% or higher will be considered as clouds.
CLD_PRB_THRESH = 40
# This parameter is used to set the threshold for dark NIR pixels.
# The value is set to 0.15, meaning that pixels with a NIR value of 0.15 or lower will be considered as dark pixels.
NIR_DRK_THRESH = 0.15
# This parameter is used to set the distance for cloud projection.
# The value is set to 2, meaning that the cloud projection will be 2 pixels away from the cloud.
CLD_PRJ_DIST = 2
# This parameter is used to set the buffer size for dilating the cloud-shadow mask.
# The value is set to 100, meaning that the cloud-shadow mask will be dilated by 100 pixels.
BUFFER = 100

In [68]:
s2_sr_cld_col = get_s2_sr_cld_col(aoi, start_date=start_date_str, end_date=end_date_str) # Get the S2 collection
# Apply cloud filters
s2_data = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
# Clip to area of interest
s2_data = s2_data.clip(aoi)
# Get the projection
s2Projection = ee.Image(s2_data).select('B4').projection()
# Make sure that data is in its native projection
s2_data = s2_data.setDefaultProjection(s2Projection)


### S1 Download

Same game for Sentinel-1 minus the cloud cover. We are using the VV and VH filters

In [69]:
s1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(aoi)
    .filterDate(start_date_str, end_date_str)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    .filter(ee.Filter.eq('resolution_meters', 10))
    .select(['VV', 'VH']))


s1_median = s1.median().clip(aoi)
s1_data = s1_median.setDefaultProjection(s2Projection)


### GEDI

GEDI download requires a bit more work. We apply a filter to remove all low quality measurements. 

If there are not enough measurements then increase number n_dats

In [70]:
# Number of days between input date and start state. GEDI went on a maintenance break in 2024. 
# So, if your start date is in 2024 or early 2025 and you don't have enough points, you can increase this number
n_days = 565


# Define date range
end_date_str = date_input
end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

# Range of data collection
start_date = end_date - datetime.timedelta(days=n_days)
# Convert back to string for GEE
start_date_str = start_date.strftime('%Y-%m-%d')

In [71]:
# ---- LOAD GEDI DATASET ----
# Define mask that removes low quality points
def quality_mask(im):
    return (
        im.updateMask(im.select('l4_quality_flag').eq(1))
          .updateMask(im.select('degrade_flag').eq(0))
    )

# Access data
gedi = ( 
        ee.ImageCollection("LARSE/GEDI/GEDI04_A_002_MONTHLY")
        .filterDate(start_date_str, end_date_str)
        .filter(ee.Filter.bounds(geometry)))

# Get the projection
gedi_proj = ee.Image(gedi.first().select('agbd').projection())

# Apply mask, then get the mosaic, only get the required 'agbd' Band (AboveBround Biomass Density)
gedi_mosaic = gedi.map(quality_mask).mosaic().select('agbd').setDefaultProjection(gedi_proj)
# Set the data to its native projection
gedi_mosaic = gedi_mosaic.setDefaultProjection(gedi_proj)

### World Cover

World Cover Map is straightforward

In [72]:
# Load the WorldCover dataset
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first()  # clip to 100km radius
# Use sentinel 2 projection
worldcover= worldcover.clip(aoi).setDefaultProjection(s2Projection)


### DEM

Digitial Elevation Model (DEM) includes elevation and allows for calculation of the slope

In [73]:
# Access data
dem = ee.ImageCollection("COPERNICUS/DEM/GLO30")
# Select Elevation band
dem = dem.select(['DEM']).mosaic().clip(aoi)
# Get Slope
slope = ee.Terrain.slope(dem)
# Set to sentinel 2 projection
dem_bands = dem.addBands(slope).setDefaultProjection(s2Projection)

# Plot everything together

We can have a quick look if the data looks alright and plot it. First we define a visualisation for all the data sources. THen we plot them.

You can toggle the layers on and off at the top right. Make sure all layers look correct

In [74]:
# Define RGB Visualization Parameters
rgb_vis = {
"bands": ["B4", "B3", "B2"],  # True color (Red, Green, Blue)
"min": 0, "max": 3500,  # Normalize RGB range
}

# Optional: visualize settings for map
dem_vis = {
    'min': 0,
    'max': 300,
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']
}

# Visualization: ESA WorldCover classes
worldcover_vis = {
    'min': 10,
    'max': 100,
    'palette': [
        '#006400',  # Tree cover
        '#ffbb22',  # Shrubland
        '#ffff4c',  # Grassland
        '#f096ff',  # Cropland
        '#fa0000',  # Built-up
        '#b4b4b4',  # Bare / sparse vegetation
        '#f0f0f0',  # Snow and ice
        '#0064c8',  # Permanent water bodies
        '#0096a0',  # Herbaceous wetland
        '#00cf75',  # Mangroves
        '#fae6a0',  # Moss and lichen
    ]
}

s1_vis = {'bands': ['VV'], 'min': -20, 'max': 0}


gedi_vis = {
  'min': 0,
  'max': 200,
  'palette': ['#edf8fb','#b2e2e2','#66c2a4','#2ca25f','#006d2c']}


In [75]:
m = geemap.Map()
m.set_center(center_aoi[0], center_aoi[1], 13)



m.addLayer(s2_data, rgb_vis, "RGB Sentinel-2")
m.addLayer(s1_data.select('VV'), s1_vis, "VV bands")
m.addLayer(gedi_mosaic, gedi_vis, "GEDI AGBD")
m.addLayer(worldcover.select('Map'), worldcover_vis, "ESA WorldCover 2020")
m.addLayer(dem_bands.select('DEM'), dem_vis, "DEM")

m



Map(center=[np.float64(51.109305072533786), np.float64(10.436138145236526)], controls=(WidgetControl(options=[…

# Resample to common 100m Resolution

The resolution of our data is quite high with GEDI having roughly 25m resolution, for the analysis, we get better results by resampling to a larger grid of 100m pixel size.

I am using the EPSG:3857 projection for this project as my area of interest is in Germany

In [76]:
# Example variables (you should replace these with your actual ee.Image inputs)

# Grid settings
grid_scale = 100 # You can change this to a higher (or lower) resolution, but keep it above 25
grid_projection = ee.Projection(projection_at_aoi).atScale(grid_scale)

# Combine all bands into one stacked image
stacked = s2_data.addBands(gedi_mosaic).addBands(dem_bands).addBands(worldcover).addBands(s1_data)

# Actually, we are reprojecting everything to the GEDI projection
# (The GEDI data is a bit fickle about being resampled)
stacked = stacked.setDefaultProjection(gedi_proj)

# Apply bilinear resampling
stacked = stacked.resample('bilinear')
stacked_resampled = (
    stacked
    .reduceResolution(reducer=ee.Reducer.mean(), maxPixels=1024)
    .reproject(crs=grid_projection))

# Clean up mask (remove transparency)
stacked_resampled = stacked_resampled.updateMask(stacked_resampled.mask().gt(0))



Quick check that pixels are aligning after the resampling

Zoom in on the GEDI pixels and turn off the layer to confirm that they overlap with Sentinel 2

In [77]:
m = geemap.Map()
m.set_center(center_aoi[0], center_aoi[1], 13)



m.addLayer(stacked_resampled, rgb_vis, "RGB Sentinel-2")
#m.addLayer(stacked_resampled.select('VV'), s1_vis, "VV bands")
m.addLayer(stacked_resampled.select('agbd'), gedi_vis, "GEDI AGB resampled")
#m.addLayer(stacked_resampled.select('Map'), worldcover_vis, "ESA WorldCover 2020")
m.addLayer(gedi_agbd.mosaic().clip(aoi), gedi_vis, "GEDI AGB native")

#m.addLayer(stacked_resampled.select('slope'), dem_vis, "DEM")

m



Map(center=[np.float64(51.109305072533786), np.float64(10.436138145236526)], controls=(WidgetControl(options=[…

# Download

In [78]:
bands_stacked = stacked.bandNames().getInfo()
print(bands_stacked)

['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'SCL', 'agbd', 'DEM', 'slope', 'Map', 'VV', 'VH']


In [79]:

# Define folder path
folder_path = "./data"

# Create folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

In [ ]:
# Unfortunately, Google Earth Explorer hates it when you want to download data to your local disk
# You maybe need to run this cell several times because of errors
# The download of 1 band can take anything between 10s to 10min (not depending on file size but Google server mood)
bands = bands_stacked

for i in bands:
    download_name = "{}/resampled100_{}.tif".format(folder_path,i)
    print(download_name)
    geemap.ee_export_image(stacked_resampled.select(i), filename=download_name, scale=grid_scale, file_per_band=False)

./data/resampled100_B2.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\juliu\Desktop\Coding\AGB\data\resampled100_B2.tif
./data/resampled100_B3.tif
Generating URL ...
Please wait ...


# End Part 1

After the download is complete, we are done with google earth explorer and the rest of the work can be done locally in Python and should run extremely fast.

If the download step would not take forever, this would have been 1 notebook.